In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from model import *

In [ ]:
from fastai.conv_learner import *
from fastai.dataset import *

In [ ]:
class UnsupFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        super().__init__(fnames, transform, path)
    def get_y(self, i): return None
    def get_c(self): return 0

In [ ]:
val_idxs = get_cv_idxs(len(fnames), val_pct=min(0.01/keep_pct, 0.1))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(fnames), np.array(fnames))
len(val_x),len(trn_x)

In [ ]:
aug_tfms = [
    RandomDihedral(tfm_y=TfmType.NO),
    RandomLighting(b=0.05, c=0.05)]

In [ ]:
arch = f
sz_lr = 72

scale,bs = 2,64
# scale,bs = 4,32
sz_hr = sz_lr*scale

In [ ]:
def get_MD(trn, val):
    tfms = tfms_from_model(
        arch,
        sz_lr,
        tfm_y=TfmType.NO,
        aug_tfms=aug_tfms,
        sz_y=sz_hr)
    datasets = ImageData.get_ds(
        MatchedFilesDataset,
        (trn, None),
        (val, None),
        tfms, path=PATH_TRN)
    md = ImageData(
        PATH,
        datasets,
        bs, num_workers=16,
        classes=None)

In [ ]:
MD1 = get_MD(trn1, val1)
MD2 = get_MD(trn2, val2)
MD3 = get_MD(trn3, val3)

In [ ]:
class Loss(nn.Module):
    def __init__(self, scale=0.01, Tscale=0.01):
        self.appr = TriAppearanceLoss()
        self.smooth = Smooth()
        self.scale = scale
    def forward(d1, d2, d3, poses_x2, x1, x2, x3):
        return self.appr(d1, d3, poses_x2, x1, x2, x3)+
            scale * self.smooth(d1, d2, d3)
        

In [ ]:
m = TriDepth(get_base(), 1)

In [ ]:
l = Loss() 

In [ ]:
opt = optim.Adam(m.parameters())

In [ ]:
def fit(model, metric, opt, MD1, MD2, MD3):
    DL1, DL2, DL3 = iter(MD1), iter(MD2), iter(MD3)
    losses = []
    for i in tqdm(range(len(DL1))):
        opt.zero_grad()
        imgs1, imgs2, imgs3= V([next(DL1)), next(DL2),next(DL3)])
        d1, d2, d3, poses_x2 = model(imgs1, imgs2, imgs3)
        loss = metric(d1, d2, d3, poses_x2, x1, x2, x3)
        loss.backward()
        opt.step()
        losses.append(loss)
    return torch.cat(loesses).mean()

In [ ]:
fit(m, l)